<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Load-Data-and-Packages" data-toc-modified-id="Load-Data-and-Packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load Data and Packages</a></span></li><li><span><a href="#Prepare-DMatrix-for-Training" data-toc-modified-id="Prepare-DMatrix-for-Training-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prepare <code>DMatrix</code> for Training</a></span><ul class="toc-item"><li><span><a href="#Choose-columns-to-use" data-toc-modified-id="Choose-columns-to-use-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Choose columns to use</a></span></li><li><span><a href="#past/future-split" data-toc-modified-id="past/future-split-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span><code>past</code>/<code>future</code> split</a></span></li><li><span><a href="#train/val-split" data-toc-modified-id="train/val-split-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span><code>train</code>/<code>val</code> split</a></span></li><li><span><a href="#pd.DataFrame---&gt;-xgb.DMatrix" data-toc-modified-id="pd.DataFrame--->-xgb.DMatrix-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span><code>pd.DataFrame</code> --&gt; <code>xgb.DMatrix</code></a></span></li></ul></li><li><span><a href="#Test-my-XGB_Tuner-Object" data-toc-modified-id="Test-my-XGB_Tuner-Object-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Test my <code>XGB_Tuner</code> Object</a></span></li><li><span><a href="#Train-a-Master-Model" data-toc-modified-id="Train-a-Master-Model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Train a Master Model</a></span></li><li><span><a href="#Train-Many-Sub-Models" data-toc-modified-id="Train-Many-Sub-Models-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Train Many Sub-Models</a></span></li><li><span><a href="#Predict-future-dataset-row-by-row" data-toc-modified-id="Predict-future-dataset-row-by-row-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Predict <code>future</code> dataset row-by-row</a></span></li></ul></div>

## Load Data and Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import xgboost as xgb
from bayes_opt import BayesianOptimization

In [2]:
import pickle
with open('scratch/variables.pkl', 'rb') as f:
    (cat_cols, num_cols, bin_cols, id_col, target_col, 
    cat2int_cols, cat2int_encoders, cat2onehot_cols, 
    cat2onehot_sparse) = pickle.load(f)

In [3]:
full = pd.read_pickle('scratch/full_engineered.pkl')

In [4]:
display(full.info())
display(full.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284127 entries, 0 to 284126
Data columns (total 53 columns):
dataset                      284127 non-null object
air_store_id                 284127 non-null object
id                           284127 non-null object
visit_date                   284127 non-null datetime64[ns]
visitors                     284127 non-null int64
weekday                      284127 non-null int64
year                         284127 non-null int64
month                        284127 non-null int64
day_of_year                  284127 non-null int64
days_in_month                284127 non-null int64
week_of_year                 284127 non-null int64
is_month_end                 284127 non-null bool
days_since_20160101          284127 non-null int64
holiday_flg                  284127 non-null int64
tomorrow_is_holiday          284127 non-null int32
yesterday_is_holiday         284127 non-null int32
air_genre_name               284127 non-null object
air_area_n

None

,dataset,air_store_id,id,visit_date,visitors,weekday,year,month,day_of_year,days_in_month,...,air_genre_name_int,air_area_name_int,hpg_genre_name_int,hpg_area_name_int,Todofuken_int,city_int,street_int,area_genre_int,store_weekday_int,store_weekday_holiday_int
0,past,air_ba937bf13d40fb24,air_ba937bf13d40fb24_2016-01-13,2016-01-13,25,2,2016,1,13,31,...,4,62,16,31,7,29,77,144,4222,7860
1,past,air_ba937bf13d40fb24,air_ba937bf13d40fb24_2016-01-14,2016-01-14,32,3,2016,1,14,31,...,4,62,16,31,7,29,77,144,4223,7862
2,past,air_ba937bf13d40fb24,air_ba937bf13d40fb24_2016-01-15,2016-01-15,29,4,2016,1,15,31,...,4,62,16,31,7,29,77,144,4224,7864
3,past,air_ba937bf13d40fb24,air_ba937bf13d40fb24_2016-01-16,2016-01-16,22,5,2016,1,16,31,...,4,62,16,31,7,29,77,144,4225,7866
4,past,air_ba937bf13d40fb24,air_ba937bf13d40fb24_2016-01-18,2016-01-18,6,0,2016,1,18,31,...,4,62,16,31,7,29,77,144,4220,7858


## Prepare `DMatrix` for Training

### Choose columns to use

In [5]:
use_cols = num_cols + bin_cols + cat2int_cols
print(use_cols)

['weekday', 'year', 'month', 'day_of_year', 'days_in_month', 'week_of_year', 'days_since_20160101', 'latitude_air', 'longitude_air', 'n_stores_same_street', 'n_stores_same_city', 'n_stores_same_Todofuken', 'n_reserves', 'n_reserve_visitors', 'reserve_avg_hours_ahead', 'mean_visitors', 'median_visitors', 'min_visitors', 'max_visitors', 'is_month_end', 'holiday_flg', 'tomorrow_is_holiday', 'yesterday_is_holiday', 'air_store_id_int', 'air_genre_name_int', 'air_area_name_int', 'hpg_genre_name_int', 'hpg_area_name_int', 'Todofuken_int', 'city_int', 'street_int', 'area_genre_int', 'store_weekday_int', 'store_weekday_holiday_int']


### `past`/`future` split

In [6]:
past = full.query('dataset=="past"')
future  = full.query('dataset=="future"')

print('Past data set:')
display(past.shape)
# display(past.head())

print('Future data set:')
display(future.shape)
# display(future.head())

Past data set:


(252108, 53)

Future data set:


(32019, 53)

### `train`/`val` split

In [7]:
train_selector = past.visit_date < '2017-02-01'
X_train = past[train_selector][use_cols]
y_train = past[train_selector][target_col].apply(np.log1p)    # apply np.log1p() (log(1+x)) to visitors count, to correct for high skewness

print('Training set dimensions...')
print('- X_train:', X_train.shape)
print('- y_train:', y_train.shape)

Training set dimensions...
- X_train: (194235, 34)
- y_train: (194235,)


In [8]:
val_selector = past.visit_date >= '2017-02-01'
X_val = past[val_selector][use_cols]
y_val = past[val_selector][target_col].apply(np.log1p)

print('Validation set dimensions...')
print('- X_val:', X_val.shape)
print('- y_val:', y_val.shape)

Validation set dimensions...
- X_val: (57873, 34)
- y_val: (57873,)


### `pd.DataFrame` --> `xgb.DMatrix`

In [9]:
train_dmatrix = xgb.DMatrix(X_train, label=y_train)
val_dmatrix = xgb.DMatrix(X_val, label=y_val)

eval_setting = [ (train_dmatrix, 'train'), (val_dmatrix, 'eval') ]

## Test my `XGB_Tuner` Object

In [10]:
from helper.tuning import XGB_Tuner

In [11]:
params_range = {'max_depth': (3,10),
                'min_child_weight': (80, 120),
                'colsample_bytree': (0.2, 0.8),
                'subsample': (0.7, 1),
                'gamma': (0, 5)
               }

In [12]:
# a = XGB_Tuner(params_range, train_dmatrix, val_dmatrix).get_tuned_params(init_points=1, n_iter=1)
# a

## Train a Master Model

## Train Many Sub-Models

In [13]:
from helper.tuning import XGB_Tuner

In [14]:
submodel = {}

In [15]:
submodel_by_features = ['weekday', 'holiday_flg', 'Todofuken_int', 'city_int']

In [16]:
for feat in submodel_by_features:
    submodel[feat] = {}
    print('Training a sub-model for each {}'.format(feat))
    for feat_value in past[feat].unique():
        print('  Sub-model for {}={}'.format(feat, feat_value))
        
        # train and val datasets are used for tuning model parameters
        train_selector = (X_train[feat] == feat_value)
        X_train_subset = X_train[train_selector]
        y_train_subset = y_train[train_selector]

        val_selector = (X_val[feat] == feat_value)
        X_val_subset = X_val[val_selector]
        y_val_subset = y_val[val_selector]

        
        # past dataset is used for training a model for prediction    
        past_selector = (full[feat]==feat_value)
        X_past = past[past_selector][use_cols]
        y_past = past[past_selector][target_col].apply(np.log1p)

        train_data = xgb.DMatrix(X_train_subset, label=y_train_subset)
        val_data = xgb.DMatrix(X_val_subset, label=y_val_subset)
        past_data = xgb.DMatrix(X_past, label=y_past)

        params = XGB_Tuner(params_range, train_data, val_data).get_tuned_params()

        submodel[feat][feat_value] = xgb.train(params=params, dtrain=past_data)

Training a sub-model for each weekday
  Sub-model for weekday=2


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ 0.00029828]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 51, 'nit': 3, 'warnflag': 2}
  " state: %s" % convergence_dict)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([  8.48355294e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 57, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b termin

  Sub-model for weekday=3


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ -4.16106462e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 49, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ -3.59077058e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 52, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00136969]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 59, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bf

  Sub-model for weekday=4


...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ -3.85739439e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 50, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ -5.76501403e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 60, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00012628]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 52, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bf

  Sub-model for city_int=47


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ -2.99413794e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 48, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ 0.00024702]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 50, 'nit': 3, 'warnflag': 2}
  " state: %s" % convergence_dict)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ -2.61220602e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 50, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bf

  Sub-model for city_int=0


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([  1.12112118e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 54, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ -1.48358740e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 58, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ -3.03518518e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 53, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin

## Predict `future` dataset row-by-row

In [17]:
def pred_rowbyrow(row):
    row_dmatrix = xgb.DMatrix(pd.DataFrame().append(row)[use_cols])
    pred = {}
    for feat in submodel_by_features:
        use_model = submodel[feat][row[feat]]
        pred_colname = 'pred_by_'+str(feat)
        pred[pred_colname] = use_model.predict(row_dmatrix)[0]
    return pd.Series(pred)

In [18]:
pred = future[use_cols].apply(pred_rowbyrow, axis=1)

In [19]:
pred = pred.join(future.id)

In [20]:
pred['visitors'] = pred.filter(regex='pred_').mean(axis=1).apply(np.expm1)

In [21]:
pred.head()

,pred_by_Todofuken_int,pred_by_city_int,pred_by_holiday_flg,pred_by_weekday,id,visitors
252108,1.068684,1.362614,1.141763,1.276435,air_00a91d42b08b08d9_2017-04-23,2.361455
252109,2.933769,3.037839,2.801515,2.901414,air_00a91d42b08b08d9_2017-04-24,17.515983
252110,3.042902,3.079802,2.869271,3.049279,air_00a91d42b08b08d9_2017-04-25,19.293758
252111,3.232437,3.233520,3.206155,3.147772,air_00a91d42b08b08d9_2017-04-26,23.654778
252112,3.338109,3.233520,3.299712,3.196580,air_00a91d42b08b08d9_2017-04-27,25.232003


In [22]:
submission = pred[['id', 'visitors']]

In [23]:
submission.to_csv('output/sub_xgb_ensemble.csv', index=False)

In [24]:
with open('scratch/submodels.pkl', 'wb') as f:
    pickle.dump(submodel, f)

In [26]:
submodel['city_int'][0]